In [1]:
import os
import pickle
import pandas as pd
from cellCnn.ms.utils.helpers import *
from cellCnn.ms.utils.helpers import calc_frequencies
from cellCnn.ms.utils.helpers import get_fitted_model
from cellCnn.utils import mkdir_p
from cellCnn.utils import save_results

def get_prepared_data(cluster_to_celltype_dict, df):
    rrms_df = df[df['diagnosis'] == 'RRMS']
    rrms_patients2df = {id: patient_df.drop(columns=['diagnosis', 'gate_source']) for id, patient_df in
                        rrms_df.groupby('gate_source')}
    nindc_df = df[df['diagnosis'] == 'NINDC']
    nindc_patients2df = {id: patient_df.drop(columns=['diagnosis', 'gate_source']) for id, patient_df in
                         nindc_df.groupby('gate_source')}
    #### here we could see freq differences across the 2 groups
    print('Creating Frequencies...')
    rrms_patients_freq = {id: calc_frequencies(patient_df, cluster_to_celltype_dict, return_list=True) for
                          id, patient_df in
                          rrms_patients2df.items()}
    nindc_patients_freq = {id: calc_frequencies(patient_df, cluster_to_celltype_dict, return_list=True) for
                           id, patient_df
                           in nindc_patients2df.items()}

    print('DONE')
    ### desease states 1 = RRMS and 0 = NINDC
    selection_pool_rrms_cd8 = [(df.loc[:, df.columns != 'cluster'], rrms_patients_freq[patient], 1)
                               for patient, df in rrms_patients2df.items()]
    selection_pool_nindc_cd8 = [(df.loc[:, df.columns != 'cluster'], nindc_patients_freq[patient], 0)
                                for patient, df in nindc_patients2df.items()]
    # make sure list are equally long:
    if len(selection_pool_rrms_cd8) > len(selection_pool_nindc_cd8):
        selection_pool_rrms_cd8 = selection_pool_rrms_cd8[:len(selection_pool_nindc_cd8)]
    elif len(selection_pool_rrms_cd8) < len(selection_pool_nindc_cd8):
        selection_pool_nindc_cd8 = selection_pool_nindc_cd8[:len(selection_pool_rrms_cd8)]
    all_chunks = selection_pool_rrms_cd8 + selection_pool_nindc_cd8
    np.random.shuffle(all_chunks)  # to get differing phenotypes...
    X = [selection[0].to_numpy() for selection in all_chunks]
    freqs = [selection[1] for selection in all_chunks]
    Y = [selection[2] for selection in all_chunks]
    print('DONE: batches created')
    return X, freqs, Y


In [3]:
cytokines = ['CCR2', 'CCR4', 'CCR6', 'CCR7', 'CXCR4', 'CXCR5', 'CD103', 'CD14', 'CD20', 'CD25', 'CD27', 'CD28',
             'CD3',
             'CD4', 'CD45RA', 'CD45RO', 'CD56', 'CD57', 'CD69', 'CD8', 'TCRgd', 'PD.1', 'GM.CSF', 'IFN.g', 'IL.10',
             'IL.13', 'IL.17A', 'IL.2', 'IL.21', 'IL.22', 'IL.3', 'IL.4', 'IL.6', 'IL.9', 'TNF.a']
infile = 'cohort_denoised_clustered_diagnosis_patients.csv'
indir = 'data/input'
outdir = 'stl_cv_filters'
rand_seed = 123
train_perc = 0.7
test_perc = 0.3
batch_size_pheno = 840  # deprecated  # so a size of 8425 is about equally sized in batches
batch_size_cd4 = 550  # deprecated #so a size of 550 gets me 16 batches for cd4
## information from ms_data project
cluster_to_celltype_dict = {0: 'b', 1: 'cd4', 3: 'nkt', 8: 'cd8', 10: 'nk', 11: 'my', 16: 'dg'}

np.random.seed(rand_seed)
mkdir_p(outdir)
df = pd.read_csv(os.path.join(indir, infile), index_col=0)
df = df.drop_duplicates()
print('Read in the data')
X, freqs, Y = get_prepared_data(cluster_to_celltype_dict, df)
freq_idx =3
cd8 = [series[freq_idx] for series in freqs]
X_test, X_train, X_valid, cd8_test, cd8_train, cd8_valid, y_test, y_train, y_valid = split_test_train_valid(
    X, cd8, Y,
    train_perc=train_perc,
    test_perc=test_perc,
    valid_perc=0.5)

Read in the data
Creating Frequencies...
DONE
DONE: batches created


In [4]:
print('Filter CV')
accs = dict()
nfilters_max = 20
for i in range(2, nfilters_max):
    outdir = f'filter_cv_stl/mtl_class_c8_{i}_filters'
    model = get_fitted_model(X_train, X_valid, y_train, y_valid,
                             nsubset=100, ncell=2000,
                             quant_normed=True, scale=False,
                             nfilters=[i], max_epochs=20,
                             learning_rate=None, nrun=3,
                             per_sample=True, regression=False,
                             maxpool_percentages=[0.5, 1.0, 2.5, 5.0, 10.0],
                             outdir=outdir, verbose=True, mtl_tasks=1)
    results = model.results
    accs[i] = results['accuracies']
    pickle.dump(results, open(os.path.join(outdir, 'results.pkl'), 'wb'))
    save_results(results, outdir, cytokines)
    print(f'DONE building model {i}')
print('DONE building models')
np.save('resulting_accs.npy', accs)

Filter CV


2022-04-06 16:25:09.245410: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 data_input (InputLayer)     [(None, 2000, 35)]        0         
                                                                 
 conv1 (Conv1D)              (None, 2000, 2)           72        
                                                                 
 lambda (Lambda)             (None, 2)                 0         
                                                                 
 dropout (Dropout)           (None, 2)                 0         
                                                                 
 phenotype_pred (Dense)      (None, 2)                 6         
                                                                 
Total params: 78
Trainable params: 78
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20


2022-04-06 16:25:09.528603: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-04-06 16:25:09.528620: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-04-06 16:25:09.529079: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.


1/1 [==============================] - 2s 2s/step - loss: 0.6931 - accuracy: 0.5682 - precision: 0.6667 - recall: 0.5000 - val_loss: 0.6953 - val_accuracy: 0.1111 - val_precision: 0.1111 - val_recall: 0.1111
Epoch 2/20
1/1 [==============================] - 0s 61ms/step - loss: 0.6929 - accuracy: 0.5455 - precision: 0.5455 - recall: 0.5455 - val_loss: 0.6973 - val_accuracy: 0.1111 - val_precision: 0.1111 - val_recall: 0.1111
Epoch 3/20
1/1 [==============================] - 0s 50ms/step - loss: 0.6927 - accuracy: 0.5455 - precision: 0.5455 - recall: 0.5455 - val_loss: 0.6993 - val_accuracy: 0.1111 - val_precision: 0.1111 - val_recall: 0.1111
Epoch 4/20
1/1 [==============================] - 0s 49ms/step - loss: 0.6926 - accuracy: 0.5455 - precision: 0.5455 - recall: 0.5455 - val_loss: 0.7012 - val_accuracy: 0.1111 - val_precision: 0.1111 - val_recall: 0.1111
Epoch 5/20
1/1 [===========

...

1/1 [==============================] - 0s 143ms/step - loss: 0.6927 - accuracy: 0.5455 - precision_52: 0.5455 - recall_52: 0.5455 - val_loss: 0.6979 - val_accuracy: 0.1111 - val_precision_52: 0.1111 - val_recall_52: 0.1111
Epoch 5/20
1/1 [==============================] - 0s 151ms/step - loss: 0.6929 - accuracy: 0.5455 - precision_52: 0.5455 - recall_52: 0.5455 - val_loss: 0.6994 - val_accuracy: 0.1111 - val_precision_52: 0.1111 - val_recall_52: 0.1111
Epoch 6/20
1/1 [==============================] - 0s 438ms/step - loss: 0.6928 - accuracy: 0.7778 - precision_52: 0.7778 - recall_52: 0.7778
Model: "model_53"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 data_input (InputLayer)     [(None, 2000, 35)]        0         
                     

2022-04-06 16:28:13.219235: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-04-06 16:28:13.219262: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-04-06 16:28:13.220081: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.


1/1 [==============================] - 1s 1s/step - loss: 0.6930 - accuracy: 0.5455 - precision_53: 0.5455 - recall_53: 0.5455 - val_loss: 0.6943 - val_accuracy: 0.1111 - val_precision_53: 0.1111 - val_recall_53: 0.1111
Epoch 2/20
1/1 [==============================] - ETA: 0s - loss: 0.6928 - accuracy: 0.6591 - precision_53: 0.6591 - recall_53: 0.6591

2022-04-06 16:28:14.761505: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-04-06 16:28:14.761532: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-04-06 16:28:14.942942: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2022-04-06 16:28:14.944159: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-04-06 16:28:14.945894: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: filter_cv_stl/mtl_class_c8_19_filters/stats/tensorboard/model_53_1649255293.219061/train/plugins/profile/2022_04_06_16_28_14

2022-04-06 16:28:14.947667: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to filter_cv_stl/mtl_class_c8_19_filters/stats/tensorboard/model_53_1649255293.219061/train/plugins/profile/2022_04_06_16_28_14/Eliass-MacBook-Pro.local.trace.json.gz
2022-04-06 16:28

1/1 [==============================] - 0s 231ms/step - loss: 0.6928 - accuracy: 0.6591 - precision_53: 0.6591 - recall_53: 0.6591 - val_loss: 0.6956 - val_accuracy: 0.1111 - val_precision_53: 0.1111 - val_recall_53: 0.1111
Epoch 3/20
1/1 [==============================] - 0s 140ms/step - loss: 0.6924 - accuracy: 0.5909 - precision_53: 0.5909 - recall_53: 0.5909 - val_loss: 0.6969 - val_accuracy: 0.1111 - val_precision_53: 0.1111 - val_recall_53: 0.1111
Epoch 4/20
1/1 [==============================] - 0s 137ms/step - loss: 0.6935 - accuracy: 0.5455 - precision_53: 0.5455 - recall_53: 0.5455 - val_loss: 0.6981 - val_accuracy: 0.1111 - val_precision_53: 0.1111 - val_recall_53: 0.1111
Epoch 5/20
1/1 [==============================] - 0s 136ms/step - loss: 0.6930 - accuracy: 0.5455 - precision_53: 0.5455 - recall_53: 0.5455 - val_loss: 0.6993 - val_accuracy: 0.1111 - val_precision_53: 0.1111 - val_recall_53: 0.1111
Epo